In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

/tmp/ipykernel_44785/912229180.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
from helper_code import *
import numpy as np, scipy as sp, scipy.stats, os, sys, joblib
from sklearn.impute import SimpleImputer
#from sklearn.ensemble import RandomForestClassifier
from models import *
from get_feature import *
from Generator0 import *
import pickle as pk
from tensorflow.keras.callbacks import LearningRateScheduler, ModelCheckpoint


import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
sys.path.insert(0,'lucashnegri-peakutils-51a679cd8428')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *
from keras.preprocessing import sequence
import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
import peakutils
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
!nvidia-smi

Thu Mar 14 11:51:58 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:1A:00.0 Off |                    0 |
| 30%   30C    P8    28W / 300W |  45619MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:1B:00.0 Off |                    0 |
| 30%   

In [4]:
# Setting the GPU.
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

In [5]:
# Data folder path
train_folder =  '/Data/hmd/data_split/murmur/train/'
test_folder = '/Data/hmd/data_split/murmur/test/'

In [6]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

# PI feature

In [7]:
model_folder = 'pi_interval_develop'
output_folder = '/Data/jk_paper/JK_paper/out_pi_develop'

In [8]:
per_sec=4000
winlen = 512
hoplen = 256
nmel = 120 
nsec = 20 
trim = 1 
use_mel=True
use_cqt = False 
use_stft = False
use_interval=True
maxlen1 = 120000
min_dist = 630     #np.random.choice(list(range(10,2000 ,10)))
max_interval_len = 130 #np.random.choice(list(range(100,300 ,10)))

params_feature = {'samp_sec': nsec,
              #### melspec, stft  
              'pre_emphasis': 0,
              'hop_length': hoplen,
              'win_length': winlen,
              'n_mels': nmel,
              #### cqt   
              'filter_scale': 1,
              'n_bins': 80,
              'fmin': 10,
              'maxlen1': maxlen1,
              'min_dist':min_dist,
              'max_interval_len' : max_interval_len,
              'trim' :1,
              'use_mel' : use_mel,
              'use_cqt' : use_cqt,
              'use_stft' : use_stft,
              'use_interval' : use_interval,
              'per_sec' : per_sec}


In [55]:
features_trn, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape = get_features_3lb_all_ord(train_folder, patient_files_trn, **params_feature)

  0%|                                                                                                                                                                                                                                                                                                        | 0/751 [00:00<?, ?it/s]

94391
52215
59959


  0%|▍                                                                                                                                                                                                                                                                                               | 1/751 [00:01<21:58,  1.76s/it]

106743
72631
81015
91575


  0%|▊                                                                                                                                                                                                                                                                                               | 2/751 [00:03<21:40,  1.74s/it]

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 750/751 [19:31<00:01,  1.12s/it]

73015
79351
79415


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 751/751 [19:32<00:00,  1.56s/it]

78583
melspec:  120 313
cqt:  1 1
stft:  1 1


interval:  130


In [72]:
features_test, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape = get_features_3lb_all_ord(test_folder, patient_files_test, **params_feature)

  0%|                                                                                                                                                                                                                                                                                                        | 0/191 [00:00<?, ?it/s]

46967
79159
72631


  1%|█▌                                                                                                                                                                                                                                                                                              | 1/191 [00:01<04:38,  1.47s/it]

61111
57975


  1%|███                                                                                                                                                                                                                                                                                             | 2/191 [00:02<03:18,  1.05s/it]

...

79799
92599
79543
82999


 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 190/191 [04:23<00:01,  1.35s/it]

77559


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 191/191 [04:23<00:00,  1.38s/it]

79991
melspec:  120 313
cqt:  1 1
stft:  1 1
interval:  130


In [74]:
# save feature to pickle
import pickle
with open('/Data/jk_paper/JK_paper/features_trn_pi_develop.pkl','wb') as f:
    pickle.dump((features_trn, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape) ,f)

with open('/Data/jk_paper/JK_paper/features_test_pi_develop.pkl','wb') as f:
    pickle.dump((features_test, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape),f)
    pickle.dump((features_trn, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape) ,f)



In [11]:
with open('/Data/jk_paper/JK_paper/features_trn_pi_develop.pkl', 'rb') as f:
    data1 = pickle.load(f)

with open('/Data/jk_paper/JK_paper/features_test_pi_develop.pkl', 'rb') as f:
    data2 = pickle.load(f)

In [12]:
np.array(data1).shape

/tmp/ipykernel_44785/3238672630.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(data1).shape


(5,)

In [82]:
np.array(data2).shape

/tmp/ipykernel_2448666/2796701488.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np.array(data2).shape


(5,)

In [83]:
data1

({'id': array(['2530', '2530', '2530', ..., '85349', '85349', '85349'],
        dtype='<U5'),
  'age': array([[0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0],
         ...,
         [0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 1],
         [0, 0, 0, 0, 0, 1]]),
  'sex': array([[1, 0],
         [1, 0],
         [1, 0],
         ...,
         [1, 0],
         [1, 0],
         [1, 0]]),
  'hw': array([[ 98.   ,  15.9  ],
         [ 98.   ,  15.9  ],
         [ 98.   ,  15.9  ],
         ...,
         [110.846,  23.767],
         [110.846,  23.767],
         [110.846,  23.767]]),
  'preg': array([False, False, False, ...,  True,  True,  True]),
  'loc': array([[1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.],
         ...,
         [1., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0.],
         [0., 0., 0., 1., 0.]]),
  'mel1': array([[[[-15.12594986],
           [-14.65352917],
           [-14.39900875],
           ...,
        

In [84]:
data2

({'id': array(['33151', '33151', '33151', '33151', '38337', '38337', '40798',
         '40798', '40798', '40798', '49558', '49558', '49572', '49572',
         '49572', '49607', '49628', '49628', '49628', '49628', '49641',
         '49641', '49641', '49641', '49653', '49653', '49653', '49653',
         '49683', '49683', '49683', '49683', '49691', '49691', '49691',
         '49691', '49761', '49761', '49824', '49824', '49824', '49829',
         '49829', '49829', '49829', '49832', '49832', '49896', '49896',
         '49896', '49896', '49952', '49952', '49952', '49952', '49952',
         '49978', '49980', '49980', '49980', '49980', '49983', '49983',
         '49983', '49983', '50006', '50006', '50012', '50026', '50026',
         '50047', '50047', '50047', '50049', '50049', '50049', '50049',
         '50067', '50067', '50067', '50075', '50075', '50079', '50079',
         '50079', '50086', '50086', '50086', '50104', '50104', '50104',
         '50104', '50105', '50105', '50105', '50143', '501

In [85]:
features_trn, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape=data1
features_test, mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape=data2 

In [86]:
features_trn

{'id': array(['2530', '2530', '2530', ..., '85349', '85349', '85349'],
       dtype='<U5'),
 'age': array([[0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        ...,
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1]]),
 'sex': array([[1, 0],
        [1, 0],
        [1, 0],
        ...,
        [1, 0],
        [1, 0],
        [1, 0]]),
 'hw': array([[ 98.   ,  15.9  ],
        [ 98.   ,  15.9  ],
        [ 98.   ,  15.9  ],
        ...,
        [110.846,  23.767],
        [110.846,  23.767],
        [110.846,  23.767]]),
 'preg': array([False, False, False, ...,  True,  True,  True]),
 'loc': array([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        ...,
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.]]),
 'mel1': array([[[[-15.12594986],
          [-14.65352917],
          [-14.39900875],
          ...,
          [ -8.60621357],
          [ -8.93

In [87]:
features_test

{'id': array(['33151', '33151', '33151', '33151', '38337', '38337', '40798',
        '40798', '40798', '40798', '49558', '49558', '49572', '49572',
        '49572', '49607', '49628', '49628', '49628', '49628', '49641',
        '49641', '49641', '49641', '49653', '49653', '49653', '49653',
        '49683', '49683', '49683', '49683', '49691', '49691', '49691',
        '49691', '49761', '49761', '49824', '49824', '49824', '49829',
        '49829', '49829', '49829', '49832', '49832', '49896', '49896',
        '49896', '49896', '49952', '49952', '49952', '49952', '49952',
        '49978', '49980', '49980', '49980', '49980', '49983', '49983',
        '49983', '49983', '50006', '50006', '50012', '50026', '50026',
        '50047', '50047', '50047', '50049', '50049', '50049', '50049',
        '50067', '50067', '50067', '50075', '50075', '50079', '50079',
        '50079', '50086', '50086', '50086', '50104', '50104', '50104',
        '50104', '50105', '50105', '50105', '50143', '50143', '50145',
